<a href="https://colab.research.google.com/github/Jonathan-C-Barrett/GEOG5990M/blob/main/GEOG5003M_Final_Project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5003M Final Assignment (Template)

Student ID number:

GitHub repo link:

Word count limit= 1,500 words max (markdown cells only, excluding readme)

In [ ]:
# read in required packages

import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

import matplotlib.pyplot as plt

!pip install mapclassify

In [ ]:
#Read in Health Services Data
ahah = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/ahah_v4.csv')
ahah.head()

In [ ]:
#Select only columns relating to Health Services
health = ahah[['LSOA21CD', 'ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h']]
health.head()

In [ ]:
#check for missing values
health.isna().sum()

In [ ]:
#Read in Median Property Sales
m_sales = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/land-registry-house-prices-LSOA.csv')
m_sales.head()

In [ ]:
#Select only columns relating to 2018
prop_16 = m_sales[['Code', 'Area','Year ending Mar 2016','Year ending Jun 2016', 'Year ending Sep 2016', 'Year ending Dec 2016']]
prop_16.head()

In [ ]:
#check for missing values
prop_16.isna().sum()

In [ ]:
#remove NA values
prop_16 = prop_16.dropna()
prop_16.isna().sum()

In [ ]:
#examine property info for count and data type
prop_16.info()

In [ ]:
#add conversion to LSOA21
LSOA21 = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Best_Fit_Lookup_for_EW_(V2).csv')
LSOA21.head()


In [ ]:
#inspect info for data type
LSOA21_prop.info()

In [ ]:
#combine prop_16 data and LSOA21 by code in prop_16 and LSOA11CD in LSOA21 in to remove areas outside london and make comparable
prop_16_LSOA21 = pd.merge(prop_16, LSOA21_prop, left_on='Code', right_on='LSOA11CD', how='left')
prop_16_LSOA21.head()

In [ ]:
#check Info for count
prop_16_LSOA21.info()

In [ ]:
#combine health and property data by LSOA code in property to remove areas outside london
health_prop = pd.merge(health, prop_16_LSOA21, left_on='LSOA21CD', right_on='LSOA21CD', how='right')
health_prop.head()

In [ ]:
#check info for data frame to check count and data type
health_prop.info()

In [ ]:
#check for missing values
health_prop.isnull().sum()

In [ ]:
#remove duplicated columns from Data base
health_prop_clean = health_prop.drop(columns=['Code', 'Area', 'LSOA11NM', 'LSOA11CD', 'ObjectId','LAD22NMW'])
health_prop_clean.head()


In [70]:
# Data downloaded from https://geoportal.statistics.gov.uk/datasets/f23c50d2bffa4a8d9693bcf4ed72bdaf_0/explore?location=0.278866%2C-2.487028%2C0.00
shp =gpd.read_file('/content/Output_Areas_2021_EW_BFE_V9_-6709771492190738846.geojson')


In [ ]:
#inspect data
shp.head()

In [ ]:
#combine boundary data and health and property data through LSOA code
ldn_prop = pd.merge(shp, prop_16, left_on='LSOA21CD', right_on='Code', how='right')
ldn_prop.head()

In [ ]:
#inspect data
ldn_prop.isnull().sum()

In [76]:
ldn_health_prop.to_file('London.geojson')

In [ ]:
ldn_health_prop.explore()

In [81]:
#select only london boroughs
london_shp= shp.loc[shp['LSOA21NM'].str.contains('City of London') |
                    shp['LSOA21NM'].str.contains('Barking and Dagenham')|
                    shp['LSOA21NM'].str.contains('Barnet')|
                    shp['LSOA21NM'].str.contains('Bexley')|
                    shp['LSOA21NM'].str.contains('Brent') |
                    shp['LSOA21NM'].str.contains('Bromley')|
                    shp['LSOA21NM'].str.contains('Camden')|
                    shp['LSOA21NM'].str.contains('Croydon')|
                    shp['LSOA21NM'].str.contains('Ealing')|
                    shp['LSOA21NM'].str.contains('Enfield')|
                    shp['LSOA21NM'].str.contains('Greenwich')|
                    shp['LSOA21NM'].str.contains('Hackney')|
                    shp['LSOA21NM'].str.contains('Hammersmith and Fulham')|
                    shp['LSOA21NM'].str.contains('Haringey')|
                    shp['LSOA21NM'].str.contains('Harrow')|
                    shp['LSOA21NM'].str.contains('Havering')|
                    shp['LSOA21NM'].str.contains('Hillingdon')|
                    shp['LSOA21NM'].str.contains('Hounslow')|
                    shp['LSOA21NM'].str.contains('Islington')|
                    shp['LSOA21NM'].str.contains('Kensington and Chelsea')|
                    shp['LSOA21NM'].str.contains('Kingston upon Thames')|
                    shp['LSOA21NM'].str.contains('Lambeth')|
                    shp['LSOA21NM'].str.contains('Lewisham')|
                    shp['LSOA21NM'].str.contains('Merton')|
                    shp['LSOA21NM'].str.contains('Newham')|
                    shp['LSOA21NM'].str.contains('Redbridge')|
                    shp['LSOA21NM'].str.contains('Richmond upon Thames')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Sutton')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Tower Hamlets')|
                    shp['LSOA21NM'].str.contains('Waltham Forest')|
                    shp['LSOA21NM'].str.contains('Wandsworth')|
                    shp['LSOA21NM'].str.contains('Westminster'),:]

In [84]:
#investigate data
london_shp.isnull().sum()

,0
FID,0
OA21CD,0
LSOA21CD,0
LSOA21NM,0
LSOA21NMW,0
BNG_E,0
BNG_N,0
LAT,0
LONG,0
GlobalID,0


In [ ]:
#explore on leaflet map
london_shp.explore()

In [ ]:
#check for missing values
london_shp.isna().sum()

This is an example block of mardown text I want to reference <a href="#ref1">[1]</a>. I might need to add some more citations <a href="#ref2">[2]</a><a href="#ref2">[3]</a>



Data Visualisation
- Don't forget to present your final two data visualisation (one spatial and one non-spatial) and the justifications about the decisions you made whilst preparing and visualising the data.

## References
<p><a href="add_url_here">[1]</a>Footnote citation goes here</p>
<p><a href="https://github.com/FrancescaPontin/GEOG5990">[2]</a>GEOG5990M course materials</p>
<p><a href="add_url_here">[3]</a>Citation 3</p>